## Objectives of the practical work

The objective is to get hands on experience on the fundamental elements of neural networks:
 
 - perceptron architecture (linear regression)
 - loss function
 - empirical loss
 - gradient descent

For this we will implement from scratch the data-structure and algorithms to train a perceptron. Note that slides related to the perceptron and neural networks in general are available on [moodle](https://moodle.insa-toulouse.fr/course/view.php?id=1822#section-2).

## Dataset

The objective of the regression is the prediction of the hydrodynamic performance of sailing yachts from dimensions and velocity.
The **inputs** are linked to dimension and hydrodynamics characteristics:
1. Longitudinal position of the center of buoyancy
(*flottabilité*), adimensional.
2. Prismatic coefficient, adimensional.
3. Length-displacement ratio, adimensional.
4. Beam -draught ratio ((*tiran d’eau*), adimensional.
5. Length-beam ratio, adimensional.
6. Froude number, adimensional

**Target value/predicted value (Output)** = Residuary resistance per unit weight of
displacement, adimensional

In [1]:
# Import some useful libraries and functions

import numpy as np
import pandas

def print_stats(dataset):
    """Print statistics of a dataset"""
    print(pandas.DataFrame(dataset).describe())


In [2]:
# Download the data set and place in the current folder (works on linux only)
filename = 'yacht_hydrodynamics.data'

import os.path
import requests

if not os.path.exists(filename):
    print("Downloading dataset...")
    r = requests.get('https://arbimo.github.io/tp-supervised-learning/tp2/' + filename)
    open(filename , 'wb').write(r.content)
    
print('Dataset available')

Dataset available


### Explore the dataset

- how many examples are there in the dataset?
- how many features for each example?
- what is the ground truth of the 10th example

In [3]:
# loads the dataset and slip between inputs (X) and ground truth (Y)
dataset = np.genfromtxt("yacht_hydrodynamics.data", delimiter='')
X = dataset[:, :-1] # examples features
Y = dataset[:, -1]  # ground truth
print("des exemples y en a: "+str(len(X))) #y a donc 308 exemples
# Print the first 5 examples
print_stats(X)
for i in range(0,10):
    print(f"f({X[i]}) = {Y[i]}")
    if (i==9):
        print(f"Le ground truth du dixième c'est {Y[i]}")
        
#des features y en a 6

des exemples y en a: 308
                0           1           2           3           4           5
count  308.000000  308.000000  308.000000  308.000000  308.000000  308.000000
mean    -2.381818    0.564136    4.788636    3.936818    3.206818    0.287500
std      1.513219    0.023290    0.253057    0.548193    0.247998    0.100942
min     -5.000000    0.530000    4.340000    2.810000    2.730000    0.125000
25%     -2.400000    0.546000    4.770000    3.750000    3.150000    0.200000
50%     -2.300000    0.565000    4.780000    3.955000    3.150000    0.287500
75%     -2.300000    0.574000    5.100000    4.170000    3.510000    0.375000
max      0.000000    0.600000    5.140000    5.350000    3.640000    0.450000
f([-5.    0.6   4.78  4.24  3.15  0.35]) = 8.62
f([-5.     0.565  4.77   3.99   3.15   0.15 ]) = 0.18
f([-2.3    0.565  4.78   5.35   2.76   0.15 ]) = 0.29
f([-5.     0.6    4.78   4.24   3.15   0.325]) = 6.2
f([0.    0.53  4.78  3.75  3.15  0.175]) = 0.59
f([-2.3    0.568

The following command adds a column to the inputs.

- what is in the value added this column?
- why are we doing this?

On insère une colonne de 1 au début
Cela correspond à la constante w0 nécessaire pour la suite. C'est la valeur d'initialisation, elle sera modifiée ensuite au fil de l'algorithme.

In [4]:
X = np.insert(X, 0, np.ones((len(X))), axis= 1)
print_stats(X)


           0           1           2           3           4           5  \
count  308.0  308.000000  308.000000  308.000000  308.000000  308.000000   
mean     1.0   -2.381818    0.564136    4.788636    3.936818    3.206818   
std      0.0    1.513219    0.023290    0.253057    0.548193    0.247998   
min      1.0   -5.000000    0.530000    4.340000    2.810000    2.730000   
25%      1.0   -2.400000    0.546000    4.770000    3.750000    3.150000   
50%      1.0   -2.300000    0.565000    4.780000    3.955000    3.150000   
75%      1.0   -2.300000    0.574000    5.100000    4.170000    3.510000   
max      1.0    0.000000    0.600000    5.140000    5.350000    3.640000   

                6  
count  308.000000  
mean     0.287500  
std      0.100942  
min      0.125000  
25%      0.200000  
50%      0.287500  
75%      0.375000  
max      0.450000  


## Creating the perceptron

![Perceptron for regression](https://arbimo.github.io/tp-supervised-learning/tp2/perceptron-regression.png)

We now want to define a perceptron, that is, a function of the form: 

$h_w(x) = w_0 + w_1 \times x_1 + \dots + w_n \times x_n$

- Complete the code snippet below to:
  - create the vector of weight `w`
  - implement the `h` function that evaluate an example based on the vector of weights
  - check if this works on a few examples

In [5]:
w = np.ones_like(X)

def h(w, x):
    s = 0
    for i in range(len(w)):
        s += x[i] * w[i]
    return s
print(h(np.array([0, 1, 7]), np.array([1, 2, 3])))

23


## Loss function

Complete the definiton of the loss function below such that, for a **single** example `x` with ground truth `y`, it returns the $L_2$ loss of $h_w$ on `x`.

In [6]:
def loss(w, x, y):
    
    return (h(w, x) - y)**2

## Empirical loss

Complete the function below to compute the empirical loss of $h_w$ on a **set** of examples $X$ with associated ground truths $Y$.

In [28]:
def emp_loss(w, X, Y):
    
    s = 0
    for i, x in enumerate(X):
        s += loss(w[i], x, Y[i])
        
    return s/len(X)
    

## Gradient update

A gradient update is of the form: $w \gets w + dw$

- Complete the function below so that it computes the $dw$ term (the 'update') based on a set of examples `(X, Y)` the step (`alpha`)

If you are not sure about the gradient computation, check out the [perceptron slides](https://moodle.insa-toulouse.fr/pluginfile.php/120391/mod_resource/content/1/cm1.pdf) on [Moodle](https://moodle.insa-toulouse.fr/course/view.php?id=1822) (in particular, slide 26). Make sure this computation is clear to you!

In [29]:
# w : vecteur des poids
# dw : vecteur des 'update' de chaque poids
def compute_update(w, X, Y, alpha):
    
    s = 0
    for i in range(len(X):
        s += (Y[i] - h(w[i], X[i]))
    
    return [alpha * x * s for x in X]

compute_update(w, X, Y, alpha = 10e-7)

[array([ 2.26192444e-06, -1.13096222e-05,  1.35715467e-06,  1.08119988e-05,
         9.59055963e-06,  7.12506199e-06,  7.91673555e-07]),
 array([ 2.26192444e-06, -1.13096222e-05,  1.27798731e-06,  1.07893796e-05,
         9.02507852e-06,  7.12506199e-06,  3.39288666e-07]),
 array([ 2.26192444e-06, -5.20242622e-06,  1.27798731e-06,  1.08119988e-05,
         1.21012958e-05,  6.24291146e-06,  3.39288666e-07]),
 array([ 2.26192444e-06, -1.13096222e-05,  1.35715467e-06,  1.08119988e-05,
         9.59055963e-06,  7.12506199e-06,  7.35125444e-07]),
 array([2.26192444e-06, 0.00000000e+00, 1.19881995e-06, 1.08119988e-05,
        8.48221666e-06, 7.12506199e-06, 3.95836777e-07]),
 array([ 2.26192444e-06, -5.20242622e-06,  1.28477308e-06,  1.08119988e-05,
         9.02507852e-06,  7.17030048e-06,  5.65481110e-07]),
 array([ 2.26192444e-06, -1.13096222e-05,  1.19881995e-06,  1.08119988e-05,
         8.48221666e-06,  7.12506199e-06,  1.01786600e-06]),
 array([ 2.26192444e-06, -1.13096222e-05,  1.277

## Gradient descent

Now implement the gradient descent algorithm that will:

- repeatedly apply an update the weights 
- stops when a max number of iterations is reached (do not consider early stopping for now)
- returns the final vector of weights

In [30]:
def descent(w_init, X, Y, alpha, max_iter):
    
    current_iter = 0
    current_w = w_init[:] # deep copy to not alter w_init
    
    while current_iter < max_iter:
        
        current_w += compute_update(current_w, X, Y, alpha)
        #print(emp_loss(current_w, X, Y))
        
        current_iter += 1
    
    return current_w

## Exploitation

You gradient descent is now complete and you can exploit it to train your perceptron.

- Train your perceptron to get a model.
- Visualize the evolution of the loss on the training set. Has it converged?
- Try training for several choices of `alpha` and `max_iter`. What seem like a reasonable choice?
- What is the loss associated with the final model?
- Is the final model the optimal one for a perceptron?


In [31]:
#w = np.empty_like(X)
# The lower alpha is, the higher max_iter has to be
w = np.zeros_like(X)
for i in range(0, 10000, 500):
    w2 = descent(w, X, Y, 10e-8, i)
    print(emp_loss(w2, X, Y))

# For 10e-7 : 500 is enough, it converges very quickly

339.2467464285712
248.7802467318107
233.09875800853877


KeyboardInterrupt: 

In [ ]:
# Code sample that can be used to visualize the difference between the ground truth and the prediction

num_samples_to_plot = 20
plt.plot(Y[0:num_samples_to_plot], 'ro', label='y')
yw = [h(w,x) for x in X]
plt.plot(yw[0:num_samples_to_plot], 'bx', label='$\hat{y}$')
plt.legend()
plt.xlabel("Examples")
plt.ylabel("f(examples)")

